# Analiza sportowa - Czyszczenie danych oraz ich przygotowanie do analizy
## Projekt realizowany w ramach przedmiotu: Metody Przetwarzania Dużych Ilości Danych
### Grupa projektowa:
+  Filip Kozak
+  Maciej Grzela
+  Paweł Owczarek
+  Szymon Tusznio

## W ramach niniejszego etapu przeprowadzone zostaną następujące operacje:
+ ### Wczytanie danych z pliku CSV
+ ### Usunięcie wpisów niepoprawnych z punktu widzenia typu danych
+ ### Usunięcie wpisów niepoprawnych z punktu widzenia wiedzy dziedzinowej
+ ### Modyfikacja formatu przechowywania wartości liczbowych
+ ### Wyliczenie wartości dla danych pustych

### Zaimportowanie wykorzystywanych bibliotek oraz konfiguracja wyświetlania zbiorów danych dla biblioteki Pandas

In [45]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from babel.numbers import parse_decimal

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 320)
css = """
.output {
    flex-direction: row;
}
"""
HTML('<style>{}</style>'.format(css))

ModuleNotFoundError: No module named 'babel'

### Wczytanie danych z pliku csv oraz zapisanie ich do struktury DataFrame dostępnej w ramach biblioteki Pandas
### Sprawdzenie dla jakich kolumn występują wartości nieokreślone

In [ ]:
transfer_df = pd.read_csv(
    'transfers-data-frame.csv',
    sep=',',
    header=0,
    index_col=0
)

display(transfer_df)
display(pd.isnull(transfer_df).sum())

### Wyświetlenie wierszy dla których występują wartości nieokreślone

In [ ]:
is_nan = transfer_df.isna()
rows_has_nan = is_nan.any(axis=1)
transfer_df[rows_has_nan][["Club", "Player", "ClubAssigned", "ClubAssignedLeagueLevel", "ClubAssignedLeagueNationality", "TransferValue"]]

### Wnioskując na podstawie wierszy zawierających wartości nieokreślone możemy zauważyć, że przeprowadzane transfery dotyczą:
+ ### Przenosin zawodnika do drużyny rezerw/drużyny młodzieżowej/szkółki piłkarskiej
+ ### Przenosin zawodnika do drużyny na poziomie rozgrywkowym nieuwzględnianym w ramach grupy poziomów rozgrywkowych futbolu profesjonalnego
+ ### Dyskwalifikacji/pauzy zawodnika
+ ### Przenosin zawodnika do klubu piłkarskiego, którego dane nie zostały wprowadzone
+ ### Zakończenia współpracy pomiędzy zawodnikiem a klubem

### Obserwacje spełniające jedno z czterech pierwszych kryteriów powyższej listy możemy uznać jako obserwacje odstające. Wiersze te, zostaną usunięte ze zbioru danych.
### Obserwacje spełniające piąte kryterium należy uznać jako poprawne. Aby uwzględnić przypadek zakończenia współpracy pomiędzy zawodnikiem a klubem należy stworzyć nową kolumnę wartości True/False

### Zdefiniowanie kolumny 'PlayerWithoutClub' przyjmującej wartość true dla zawodników bez klubu po transferze
### Wyczyszczenie wartości dla kolumny 'ClubAssigned'

In [34]:
without_club_label = "Bez Klubu"
transfer_df["ClubAssigned"].replace({"Bez klubuBez klubu": without_club_label}, inplace=True)

transfer_df['PlayerWithoutClub'] = transfer_df['ClubAssigned'].map(lambda x: x == without_club_label)

transfer_df[rows_has_nan][["Club", "Player", "ClubAssigned", "PlayerWithoutClub", "ClubAssignedLeagueLevel", "ClubAssignedLeagueNationality", "TransferValue"]]

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/home/maciek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/maciek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Club,Player,ClubAssigned,PlayerWithoutClub,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue
36,Legia Warszawa,Dominik Nagy,Bez Klubu,True,,,-
37,Legia Warszawa,William Rémy,Bez Klubu,True,,,-
100,Piast Gliwice,Michal Chrapek,Bez Klubu,True,,,-
186,Śląsk Wrocław,Filip Markovic,Bez Klubu,True,,,-
187,Śląsk Wrocław,Michal Chrapek,Bez Klubu,True,,,-
...,...,...,...,...,...,...,...
18966,Colorado Rapids,Oliver Larraz,Bez Klubu,True,,,-
18992,Los Angeles Galaxy,Víctor Vázquez,Bez Klubu,True,,,bez odstępnego
19014,Los Angeles Galaxy,Josh Drack,Bez Klubu,True,,,-
19015,Los Angeles Galaxy,David Bingham,Bez Klubu,True,,,-


### Wyznaczenie zbioru obserwacji do usunięcia
### Usunięcie zbioru obserwacji odstających

In [35]:
transfers_to_remove = transfer_df[rows_has_nan].loc[transfer_df['PlayerWithoutClub'] == False]
display(transfers_to_remove)

transfer_df.drop(transfers_to_remove.index.tolist(), inplace=True)

display(transfer_df)

/home/maciek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Club,ClubLeagueLevel,ClubLeagueNationality,Player,PlayerAge,PlayerPosition,PlayerPositionScut,PlayerNationality,Operation,Bought/Sold,PlayerMarketValue,ClubAssigned,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue,PlayerWithoutClub,PlayerGenericPosition


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Club,ClubLeagueLevel,ClubLeagueNationality,Player,PlayerAge,PlayerPosition,PlayerPositionScut,PlayerNationality,Operation,Bought/Sold,PlayerMarketValue,ClubAssigned,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue,PlayerWithoutClub,PlayerGenericPosition
1,Legia Warszawa,1,Polska,Ernest Muci,19,Środkowy napastnik,ŚNP,Albania,Transfer,Kupiony,"1,00 mln €",KF Tirana,1.liga,Albania,500 tys. €,False,Napastnik
2,Legia Warszawa,1,Polska,Josip Juranovic,24,Prawy obrońca,POB,Chorwacja,Transfer,Kupiony,"2,50 mln €",HNK Hajduk Split,1.liga,Chorwacja,400 tys. €,False,Obrońca
3,Legia Warszawa,1,Polska,Jasur Yakhshiboev,23,Prawy napastnik,PNP,Uzbekistan,Transfer,Kupiony,700 tys. €,Pakhtakor Tashkent,1.liga,Uzbekistan,300 tys. €,False,Napastnik
4,Legia Warszawa,1,Polska,Rafa Lopes,29,Środkowy napastnik,ŚNP,Portugalia,Transfer,Kupiony,500 tys. €,Cracovia,1.liga,Polska,150 tys. €,False,Napastnik
5,Legia Warszawa,1,Polska,Bartosz Kapustka,23,Ofensywny pomocnik,OPM,Polska,Transfer,Kupiony,"1,50 mln €",Leicester City,1.liga,Anglia,bez odstępnego,False,Pomocnik
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19015,Los Angeles Galaxy,1,Stany Zjednaczone,David Bingham,31,Bramkarz,BRK,Stany Zjednaczone,Transfer,Sprzedany,400 tys. €,Bez Klubu,,,-,True,Bramkarz
19016,Los Angeles Galaxy,1,Stany Zjednaczone,Diédié Traoré,22,Lewy obrońca,LOB,Francja,Transfer,Sprzedany,200 tys. €,Bez Klubu,,,-,True,Obrońca
19017,Los Angeles Galaxy,1,Stany Zjednaczone,Yony González,26,Prawy napastnik,PNP,Kolumbia,Wypożyczenie,Sprzedany,"1,60 mln €",Benfika Lizbona,1.liga,Portugalia,koniec wypżyczenia31 gru 2020,False,Napastnik
19018,Los Angeles Galaxy,1,Stany Zjednaczone,Carlos Harvey,20,Defensywny pomocnik,DPM,Panama,Wypożyczenie,Sprzedany,175 tys. €,Los Angeles Galaxy II,2.liga,Stany Zjednaczone,koniec wypżyczenia30 gru 2020,False,Pomocnik


### Spośród pozostałych wierszy zawierających wartości nieokreślone wszystkie posiadają dla kolumny 'PlayerWithoutClub' wartość True
### Wiersze te nie zostaną usunięte, dlatego też należy usunąć z nich wartości NaN i zastąpić je pustym ciągiem znaków

In [36]:
display(transfer_df[rows_has_nan][["Club", "Player", "ClubAssigned", "PlayerWithoutClub", "ClubAssignedLeagueLevel", "ClubAssignedLeagueNationality", "TransferValue"]])

transfer_df.fillna('', inplace=True)

/home/maciek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Club,Player,ClubAssigned,PlayerWithoutClub,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue
36,Legia Warszawa,Dominik Nagy,Bez Klubu,True,,,-
37,Legia Warszawa,William Rémy,Bez Klubu,True,,,-
100,Piast Gliwice,Michal Chrapek,Bez Klubu,True,,,-
186,Śląsk Wrocław,Filip Markovic,Bez Klubu,True,,,-
187,Śląsk Wrocław,Michal Chrapek,Bez Klubu,True,,,-
...,...,...,...,...,...,...,...
18966,Colorado Rapids,Oliver Larraz,Bez Klubu,True,,,-
18992,Los Angeles Galaxy,Víctor Vázquez,Bez Klubu,True,,,bez odstępnego
19014,Los Angeles Galaxy,Josh Drack,Bez Klubu,True,,,-
19015,Los Angeles Galaxy,David Bingham,Bez Klubu,True,,,-


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


### Jak możemy zauważyć, po wykonaniu powyższej czynności w zbiorze danych nie występują już żadne wartości nieokreślone

In [37]:
transfer_df[transfer_df.isna().any(axis=1)]

,Club,ClubLeagueLevel,ClubLeagueNationality,Player,PlayerAge,PlayerPosition,PlayerPositionScut,PlayerNationality,Operation,Bought/Sold,PlayerMarketValue,ClubAssigned,ClubAssignedLeagueLevel,ClubAssignedLeagueNationality,TransferValue,PlayerWithoutClub,PlayerGenericPosition


### Wyczyszczenie danych z kolumn 'ClubLeagueLevel' oraz 'ClubAssignedLeagueLevel'

In [38]:
display(transfer_df['ClubLeagueLevel'].value_counts())

display(transfer_df['ClubAssignedLeagueLevel'].value_counts())
transfer_df["ClubAssignedLeagueLevel"].replace({"ieżowa": "Liga Młodzieżowa", "onship": "2.liga", "rezerw": "Rezerwy klubowe"}, inplace=True)

transfer_df['ClubLeagueLevel'] = transfer_df['ClubLeagueLevel'].astype("string")
transfer_df['ClubAssignedLeagueLevel'] = transfer_df['ClubAssignedLeagueLevel'].astype("string")

transfer_df.info()

1    16087
Name: ClubLeagueLevel, dtype: Int64

1.liga              8740
2.liga              4496
3.liga              1191
                     893
4.liga               442
Liga Młodzieżowa     222
Rezerwy klubowe       93
5.liga                 8
6.liga                 2
Name: ClubAssignedLeagueLevel, dtype: Int64

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/home/maciek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/maciek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16087 entries, 1 to 19019
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Club                           16087 non-null  object
 1   ClubLeagueLevel                16087 non-null  string
 2   ClubLeagueNationality          16087 non-null  object
 3   Player                         16087 non-null  object
 4   PlayerAge                      16087 non-null  int64 
 5   PlayerPosition                 16087 non-null  object
 6   PlayerPositionScut             16087 non-null  object
 7   PlayerNationality              16087 non-null  object
 8   Operation                      16087 non-null  object
 9   Bought/Sold                    16087 non-null  object
 10  PlayerMarketValue              16087 non-null  object
 11  ClubAssigned                   16087 non-null  object
 12  ClubAssignedLeagueLevel        16087 non-null  string
 13  C

### Wyświetlenie zgrupowanych wartości dla kolumn w celu określenia, w jaki sposób powinny być czyszczone

In [39]:
display(transfer_df['ClubLeagueNationality'].value_counts())         # nie wymaga czyszczenia
display(transfer_df['PlayerAge'].value_counts())                     # nie wymaga czyszczenia
display(transfer_df['PlayerPosition'].value_counts())                # nie wymaga czyszczenia (stworzenie dodatkowej kolumny danych określających pozycję ogólną - Pomocnik, Napastnik, Obrońca, Bramkarz
display(transfer_df['PlayerNationality'].value_counts())             # nie wymaga czyszczenia
display(transfer_df['Operation'].value_counts())                     # wymaga czyszczenia (usunięcie danych niepoprawnych oraz modyfikacja formatu numerycznego, usunięcie wierszy dla wartości '?'
display(transfer_df['Bought/Sold'].value_counts())                   # zmiana wartości 'nabytki' na 'kupiony' oraz 'odejścia' na 'sprzedany'
display(transfer_df['PlayerMarketValue'].value_counts())             # modyfikacja danych do wartości liczbowych
display(transfer_df['ClubAssignedLeagueNationality'].value_counts()) # nie wymaga czyszczenia
display(transfer_df['TransferValue'].value_counts())                 # analogicznie do kolumny 'Operation'

Włochy               1886
Argentyna            1758
Brazylia             1692
Stany Zjednaczone    1460
Austria              1356
Anglia               1350
Hiszpania            1314
Portugalia           1274
Rosja                1180
Polska               1126
Holandia             1124
Francja               567
Name: ClubLeagueNationality, dtype: int64

23    1635
22    1449
21    1426
24    1410
25    1234
26    1232
20    1114
27    1050
29     927
28     878
19     803
30     596
31     481
18     417
32     404
33     305
34     210
35     149
17     131
36      70
37      62
16      54
38      32
40       8
39       6
15       4
Name: PlayerAge, dtype: int64

Środkowy napastnik     2697
Środkowy obrońca       2478
Środkowy pomocnik      1749
Bramkarz               1342
Prawy napastnik        1287
Ofensywny pomocnik     1238
Prawy obrońca          1219
Lewy napastnik         1196
Lewy obrońca           1181
Defensywny pomocnik    1162
Lewy pomocnik           199
Prawy pomocnik          159
Cofnięty napastnik      150
Pomocnik                 12
Napastnik                10
Obrońca                   8
Name: PlayerPosition, dtype: int64

Brazylia            2120
Argentyna           1850
Włochy               986
Hiszpania            986
Polska               858
                    ... 
Zambia                 2
Belize                 2
Chad                   1
Gujana Francuska       1
Zimbabwe               1
Name: PlayerNationality, Length: 133, dtype: int64

Wypożyczenie    8326
Transfer        7761
Name: Operation, dtype: int64

Sprzedany    8562
Kupiony      7525
Name: Bought/Sold, dtype: int64

200 tys. €     828
300 tys. €     822
500 tys. €     761
400 tys. €     729
250 tys. €     649
              ... 
19,00 mln €      2
80,00 mln €      2
825 tys. €       2
3,10 mln €       2
2,90 mln €       2
Name: PlayerMarketValue, Length: 120, dtype: int64

Włochy         1695
Brazylia       1478
Argentyna      1242
Anglia         1221
Hiszpania      1203
               ... 
Mozambik          2
Singapur          2
Azerbejdżan       2
Bangladesz        2
Irlandia          2
Name: ClubAssignedLeagueNationality, Length: 82, dtype: int64

wypożyczenie                         4153
bez odstępnego                       3802
-                                    1610
koniec wypżyczenia30 cze 2020         842
koniec wypżyczenia31 lip 2020         431
                                     ... 
2,90 mln €                              1
3,40 mln €                              1
750 tys. €                              1
16,70 mln €                             1
opłata za wypożyczenie:700 tys. €       1
Name: TransferValue, Length: 523, dtype: int64

### Stworzenie dodatkowej kolumny określającej ogólną pozycję zawodnika

In [40]:
transfer_df['PlayerGenericPosition'] = transfer_df['PlayerPosition'].map(lambda x: x.split(' ')[-1].capitalize())

transfer_df[['PlayerPosition', 'PlayerGenericPosition']]

/home/maciek/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PlayerPosition,PlayerGenericPosition
1,Środkowy napastnik,Napastnik
2,Prawy obrońca,Obrońca
3,Prawy napastnik,Napastnik
4,Środkowy napastnik,Napastnik
5,Ofensywny pomocnik,Pomocnik
...,...,...
19015,Bramkarz,Bramkarz
19016,Lewy obrońca,Obrońca
19017,Prawy napastnik,Napastnik
19018,Defensywny pomocnik,Pomocnik


### Wyczyszczenie danych z kolumny 'Operation'

In [41]:
transfer_df["Operation"].replace({"bez odstępnego": "Transfer", "-": "Transfer", "wypożyczenie": "Wypożyczenie", "draft": "Transfer"}, inplace=True)
transfer_df["Operation"].replace(["^koniec wypżyczenia[0-9]{1,2}[ ][a-ź]{3}[ ][0-9]{4}$"], ["Wypożyczenie"], regex=True, inplace=True)
transfer_df["Operation"].replace(["^opłata za wypożyczenie:.*$"], ["Wypożyczenie"], regex=True, inplace=True)
transfer_df["Operation"].replace(["^.*€$"], ["Transfer"], regex=True, inplace=True)

transfer_df = transfer_df[transfer_df["Operation"] != "?"]

transfer_df["Operation"].value_counts()

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


Wypożyczenie    8326
Transfer        7761
Name: Operation, dtype: int64

### Wyczyszczenie danych z kolumny 'Bought/Sold'

In [42]:
transfer_df["Bought/Sold"].replace({"Odejścia": "Sprzedany", "Nabytki": "Kupiony"}, inplace=True)

transfer_df['Bought/Sold'].value_counts()

Sprzedany    8562
Kupiony      7525
Name: Bought/Sold, dtype: int64

In [43]:
transfer_df["PlayerMarketValue"].replace({"-": 0}, inplace=True)

transfer_df['PlayerMarketValue'].value_counts()

200 tys. €     828
300 tys. €     822
500 tys. €     761
400 tys. €     729
250 tys. €     649
              ... 
19,00 mln €      2
80,00 mln €      2
825 tys. €       2
3,10 mln €       2
2,90 mln €       2
Name: PlayerMarketValue, Length: 120, dtype: int64